### carregamos o dataset

In [1]:
import pandas as pd 

resenhas = pd.read_csv('imdb-reviews-pt-br.csv')

# sera necessario limitar o numero de resenhas - 75% dos dados
resenhas = resenhas.sample(frac = 0.75).reset_index(drop=True)

troca = {
    'neg' : 0,
    'pos' : 1
}

resenhas['sentiment_b'] = resenhas.sentiment.map(troca)
resenhas = resenhas.drop(columns = ["id" , "text_en", "sentiment"], axis = 1)

### tratando o dataset

In [2]:
import nltk

nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words("portuguese")

import re

# tudo lowercase
for index, row in resenhas.iterrows():
    resenhas.at[index,'text_pt'] = row['text_pt'].lower()
    
# elimina as stop words
for index, row in resenhas.iterrows():
    # retorna apenas as palavras - eliminando os pnts de sinalizacao
    words = re.findall(r"[\w']+", row['text_pt'])
    temp = [ws for ws in words if ws not in stop_words]
    resenhas.at[index,'text_pt'] = ' '.join(temp)

# retiramos os acentos das palavras
# retiramos tb as stop words sem acento
import unidecode

for index, row in resenhas.iterrows():
    resenhas.at[index,'text_pt'] = unidecode.unidecode(row['text_pt'])
    
stop_words_sem_acentuacao = [unidecode.unidecode(sw) for sw in stop_words]

for index, row in resenhas.iterrows():
    # retorna apenas as palavras - eliminando os pnts de sinalizacao
    words = re.findall(r"[\w']+", row['text_pt'])
    temp = [ws for ws in words if ws not in stop_words_sem_acentuacao]
    resenhas.at[index,'text_pt'] = ' '.join(temp)
    
# stemizacao
nltk.download('rslp')
stemmer = nltk.RSLPStemmer()
for index, row in resenhas.iterrows():
    #retorna apenas as palavras - eliminando os pnts de sinalizacao
    words = re.findall(r"[\w']+", row['text_pt'])
    temp = [stemmer.stem(ws) for ws in words]
    resenhas.at[index,'text_pt'] = ' '.join(temp)

[nltk_data] Downloading package stopwords to /home/pedro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/pedro/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


## TF-IDF (Frequência dos Termos - Inverso da Frequência do Documento)
#### TF-IDF irá contar a frequência das palavras, ponderando-as em todo o dataset

### exemplo

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

frases = ["Assisti um filme ótimo", "Assisti um filme péssimo"]

tfidf = TfidfVectorizer(lowercase = False, max_features = 50)

caracteristicas = tfidf.fit_transform(frases)

pd.DataFrame(
    caracteristicas.todense(),
    columns = tfidf.get_feature_names()

)

,Assisti,filme,péssimo,um,ótimo
0,0.448321,0.448321,0.000000,0.448321,0.630099
1,0.448321,0.448321,0.630099,0.448321,0.000000


### aplicando ao nosso dataset 
#### substituindo o bag of words

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase = False)

caracteristicas = tfidf.fit_transform(resenhas.text_pt)

### repetimos a classificacao

In [5]:
# estratifica os dados
from sklearn.model_selection import train_test_split

SEED = 20
treino, teste, classe_treino, classe_teste = train_test_split(
    caracteristicas,
    resenhas.sentiment_b,
    random_state = SEED)

# realiza a classificacao
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(solver = 'lbfgs')
modelo.fit(treino,classe_treino)
modelo.score(teste,classe_teste)

0.8854863058011645

# Ngrams
##  É uma forma de manter uma certa sequência do texto.
#### Embora o Ngrams seja um método que pode otimizar o seu resultado devemos ter cuidado, já que ao utilizá-lo o nosso vetor Bag of Words aumenta consideravelmente de tamanho, o que também faz com que o custo computacional aumente.

#### Exemplo

In [6]:
import nltk
from nltk import tokenize
from nltk import ngrams

tokenizador = tokenize.WhitespaceTokenizer()
frase =  "Assisti um ótimo filme."
tokens = tokenizador.tokenize(frase)
pares = ngrams(tokens, 2)
list(pares)

[('Assisti', 'um'), ('um', 'ótimo'), ('ótimo', 'filme.')]

### aplicando ao dataset

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# o  vetor deve conter palavras isoladas e palavras em bigramas ( sequencia de 2 palavras)
tfidf = TfidfVectorizer(lowercase = False, ngram_range = (1,2))
caracteristicas = tfidf.fit_transform(resenhas.text_pt)

### repetimos a classificacao

In [8]:
# estratifica os dados
from sklearn.model_selection import train_test_split

SEED = 20
treino, teste, classe_treino, classe_teste = train_test_split(
    caracteristicas,
    resenhas.sentiment_b,
    random_state = SEED)

# realiza a classificacao
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(solver = 'lbfgs')
modelo.fit(treino,classe_treino)
modelo.score(teste,classe_teste)

0.8820357990079793

# Visualizando os maiores pesos ( sentimento positivo )

In [9]:
weight = pd.DataFrame(
    # peso de cada termo
    modelo.coef_[0].T,
    index = tfidf.get_feature_names()
) 
# 0 = coluna do Dataframe
weight.nlargest(10,0)

,0
otim,8.918297
excel,8.200784
perfeit,6.163989
maravilh,5.996960
favorit,5.065226
incri,4.887897
divert,4.702101
vid,4.109538
am,3.984583
brilh,3.840353


# Visualizando os menores pesos ( sentimento negativo )

In [10]:
# 0 = coluna do Dataframe
weight.nsmallest(10,0)

,0
ruim,-10.509821
pi,-10.154561
horri,-8.199204
terri,-7.760337
chat,-6.870649
nad,-6.803006
parec,-6.102259
nenhum,-5.362555
tent,-5.171855
mal,-5.038076
